<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
#ANSWER:
response.headers['Content-Type']

'application/json'

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
#ANSWER:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [5]:
#ANSWER:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)

Print the response header:

In [6]:
#ANSWER:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 18 May 2021 15:01:12 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [7]:
#ANSWER:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1621350072, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 622, \n      "risetime": 1621367739\n    }, \n    {\n      "duration": 633, \n      "risetime": 1621373538\n    }, \n    {\n      "duration": 453, \n      "risetime": 1621379496\n    }, \n    {\n      "duration": 351, \n      "risetime": 1621385459\n    }, \n    {\n      "duration": 530, \n      "risetime": 1621391270\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [8]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [9]:
#ANSWER:
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
overheads = response.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1621350072, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 622, 'risetime': 1621367739}, {'duration': 633, 'risetime': 1621373538}, {'duration': 453, 'risetime': 1621379496}, {'duration': 351, 'risetime': 1621385459}, {'duration': 530, 'risetime': 1621391270}]}


What kind of object did this give us?

In [11]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [12]:
#ANSWER:
passes = overheads['response']

In [13]:
passes[1]['risetime']

1621373538

In [14]:
x = []
for item in passes:
    x.append(item['risetime'])
x

[1621367739, 1621373538, 1621379496, 1621385459, 1621391270]

Now extract the `risetime` strings into an array called `srisetimes`:

In [15]:
#ANSWER:
srisetimes = [xpass['risetime'] for xpass in passes]
srisetimes

[1621367739, 1621373538, 1621379496, 1621385459, 1621391270]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [16]:
#ANSWER:
risetimes = [datetime.fromtimestamp(xpass['risetime']) for xpass in passes] 

In [17]:
risetimes

[datetime.datetime(2021, 5, 19, 5, 55, 39),
 datetime.datetime(2021, 5, 19, 7, 32, 18),
 datetime.datetime(2021, 5, 19, 9, 11, 36),
 datetime.datetime(2021, 5, 19, 10, 50, 59),
 datetime.datetime(2021, 5, 19, 12, 27, 50)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [18]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

19/05/21 05:55
19/05/21 07:32
19/05/21 09:11
19/05/21 10:50
19/05/21 12:27


Here is an endpoint that tells us who is onboard:

In [19]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [20]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
[print(astronaut['name']) for astronaut in astros['people']]

{'number': 7, 'message': 'success', 'people': [{'name': 'Mark Vande Hei', 'craft': 'ISS'}, {'name': 'Oleg Novitskiy', 'craft': 'ISS'}, {'name': 'Pyotr Dubrov', 'craft': 'ISS'}, {'name': 'Thomas Pesquet', 'craft': 'ISS'}, {'name': 'Megan McArthur', 'craft': 'ISS'}, {'name': 'Shane Kimbrough', 'craft': 'ISS'}, {'name': 'Akihiko Hoshide', 'craft': 'ISS'}]}
7
Mark Vande Hei
Oleg Novitskiy
Pyotr Dubrov
Thomas Pesquet
Megan McArthur
Shane Kimbrough
Akihiko Hoshide


[None, None, None, None, None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [21]:
#ANSWER:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [22]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

Response = 400: (TODO: output of lookup function here)


True

>

>

>



---



---



> > > > > > > > > © 2019 Institute of Data


---



---



